In [ ]:
%matplotlib inline

In [ ]:
import os

In [ ]:
%run notebook_setup

# Load data in from Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Transit fitting

*exoplanet* includes methods for computing the light curves transiting planets.
In its simplest form this can be used to evaluate a light curve like you would do with [batman](https://astro.uchicago.edu/~kreidberg/batman/), for example:

In [ ]:
import os
HOME = os.environ['HOME']
os.chdir(f'{HOME}/path/to/analysis')

In [ ]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import exoplanet as xo

from arctor import Arctor, info_message

In [ ]:
def instantiate_arctor(planet_name, data_dir, working_dir, file_type):
    planet = Arctor(
        planet_name=planet_name,
        data_dir=data_dir,
        working_dir=working_dir,
        file_type=file_type)

    joblib_filename = f'{planet_name}_savedict.joblib.save'
    joblib_filename = f'{working_dir}/{joblib_filename}'
    if os.path.exists(joblib_filename):
        info_message('Loading Data from Save File')
        planet.load_data(joblib_filename)
    else:
        info_message('Loading New Data Object')
        planet.load_data()

    return planet

def create_raw_lc_stddev(planet):
    ppm = 1e6
    phot_vals = planet.photometry_df
    lc_std_rev = phot_vals.iloc[planet.idx_rev].std(axis=0)
    lc_std_fwd = phot_vals.iloc[planet.idx_fwd].std(axis=0)

    lc_med_rev = np.median(phot_vals.iloc[planet.idx_rev], axis=0)
    lc_med_fwd = np.median(phot_vals.iloc[planet.idx_rev], axis=0)

    lc_std = np.mean([lc_std_rev, lc_std_fwd], axis=0)
    lc_med = np.mean([lc_med_rev, lc_med_fwd], axis=0)

    return lc_std / lc_med * ppm

In [ ]:
plot_verbose = False
save_now = False
planet_name = 'PlanetName'
file_type = 'flt.fits'

HOME = os.environ['HOME']
base_dir = os.path.join(HOME, 'path', 'to', 'base')
data_dir = os.path.join(base_dir, 'data', 'UVIS', 'MAST_2019-07-03T0738')
data_dir = os.path.join(data_dir, 'HST', 'FLTs')
working_dir = os.path.join(base_dir, 'github_analysis', 'savefiles')

In [ ]:
planet = instantiate_arctor(planet_name, data_dir, working_dir, file_type)

In [ ]:
planet.clean_cosmic_rays()
planet.calibration_trace_location()
planet.identify_trace_direction()
planet.simple_phots()
planet.center_all_traces()
planet.fit_trace_slopes()
planet.compute_sky_background(subpixels=32)
planet.compute_columnwise_sky_background()

# Run Multi-Phot

In [ ]:
# Set up the list of aperture widths and heights to search
min_aper_width = 1
max_aper_width = 100
min_aper_height = 1
max_aper_height = 100

aper_widths = np.arange(min_aper_width, max_aper_width + 2, 5)
aper_heights = np.arange(min_aper_height, max_aper_height + 2, 5)

In [ ]:
planet.do_multi_phot(aper_widths, aper_heights)

# Determine the 'best' photometry SNR

In [ ]:
planet_coarse_photometry_df = planet.photometry_df.copy()
del planet.photometry_df

In [ ]:
coarse_snr_lightcurves = create_raw_lc_stddev(planet)
coarse_min_snr = coarse_snr_lightcurves[coarse_snr_lightcurves.argmin()]
coarse_min_snr_colname = planet.photometry_df.columns[coarse_snr_lightcurves.argmin()]
coarse_min_snr_col = planet.normed_photometry_df[coarse_min_snr_colname]
coarse_temp = coarse_min_snr_colname.split('_')[-1].split('x')
coarse_min_snr_aper_width, coarse_min_snr_aper_height = np.int32(coarse_temp)

In [ ]:
info_message(f'Coarse Aperture Photometry Resulted in {coarse_min_snr:0.0f}ppm with '
             f'{coarse_min_snr_aper_width}x{coarse_min_snr_aper_height} aperture size')

In [ ]:
fine_buffer = 10
fine_aper_widths = np.arange(min_snr_aper_width - fine_buffer,
                             min_snr_aper_width + fine_buffer)

fine_aper_heights = np.arange(min_snr_aper_height - fine_buffer,
                              min_snr_aper_height + fine_buffer)

In [ ]:
planet.do_multi_phot(fine_aper_widths, fine_aper_heights)

In [ ]:
planet_fine_photometry_df = planet.photometry_df.copy()

In [ ]:
fine_snr_lightcurves = create_raw_lc_stddev(planet)
fine_min_snr = fine_snr_lightcurves[fine_snr_lightcurves.argmin()]
fine_min_snr_colname = planet.photometry_df.columns[fine_snr_lightcurves.argmin()]
fine_min_snr_flux = planet.normed_photometry_df[fine_min_snr_colname]
fine_min_snr_uncs = planet.normed_uncertainty_df[fine_min_snr_colname]
fine_temp = fine_min_snr_colname.split('_')[-1].split('x')
fine_min_snr_aper_width, fine_min_snr_aper_height = np.int32(fine_temp)

In [ ]:
info_message(f'Fine Aperture Photometry Resulted in {fine_min_snr:0.0f}ppm with '
             f'{fine_min_snr_aper_width}x{fine_min_snr_aper_height} aperture size; '
             f'with median uncertainties of {np.median(fine_min_snr_uncs)*1e6:0.0f} ppm')

# Configure system for PyMC3

In [ ]:
data_df = pd.DataFrame()
data_df['flux'] = fine_min_snr_flux.values
data_df['unc'] = fine_min_snr_uncs.values
data_df['times'] = planet.times
idx_fwd = planet.idx_fwd
idx_rev = planet.idx_rev
# data_df.sort_values('times', inplace=True)
# data_df = data_df.reset_index()
# data_df.drop(['index'], axis=1, inplace=True)
data_df

In [ ]:
# Compute a limb-darkened light curve using starry
t = data_df['times']
u = []
flux = data_df['flux']
yerr = data_df['unc']
# Note: the `eval` is needed because this is using Theano in
# the background

plt.errorbar(t[idx_fwd], flux[idx_fwd], yerr[idx_fwd], fmt='o', color="C0")
plt.errorbar(t[idx_rev], flux[idx_rev], yerr[idx_rev], fmt='o', color="C3")
plt.axhline(1.0, ls='--', color='C1')
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.xlim(t.min(), t.max());

But the real power comes from the fact that this is defined as a [Theano operation](http://deeplearning.net/software/theano/extending/extending_theano.html) so it can be combined with PyMC3 to do transit inference using Hamiltonian Monte Carlo.

## The transit model in PyMC3

In this section, we will construct a simple transit fit model using *PyMC3* and then we will fit a two planet model to simulated data.
To start, let's randomly sample some periods and phases and then define the time sampling:

In [ ]:
np.random.seed(42)

time_med = np.median(t)
med_t_diff = np.median(np.diff(t))

Then, define the parameters.
In this simple model, we'll just fit for the limb darkening parameters of the star, and the period, phase, impact parameter, and radius ratio of the planets (note: this is already 10 parameters and running MCMC to convergence using [emcee](https://emcee.readthedocs.io) would probably take at least an hour).
For the limb darkening, we'll use a quadratic law as parameterized by [Kipping (2013)](https://arxiv.org/abs/1308.0009).
This reparameterizations is implemented in *exoplanet* as custom *PyMC3* distribution :class:`exoplanet.distributions.QuadLimbDark`.

In [ ]:
from multiprocessing import cpu_count
print(f'This instance has {cpu_count()} CPUs')

In [ ]:
import pymc3 as pm
b = 0.66 # Hellier 2011
period = 0.813475  # days # exo.mast.stsci.edu
u = [0]
t0 = time_med
edepth = np.sqrt(500/1e6)

orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)
light_curves = xo.LimbDarkLightCurve(u).get_light_curve(orbit=orbit, r=edepth, t=t).eval().flatten()

In [ ]:
plt.errorbar(t, flux, yerr, fmt='o')
plt.plot(t, light_curves+1,'o')
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.xlim(t.min(), t.max());

In [ ]:
t0_planet = 55528.3684  # exo.mast.stsci.edu
n_epochs = np.int(np.round(((time_med - t0_planet) / period)-0.5))
n_epochs, ((time_med - t0_planet) / period)
t0_guess = t0_planet + (n_epochs+0.5) * period

# t0s = np.random.normal(t0_guess, 0.1*med_t_diff, size=2)
t0s = t0_guess

In [ ]:
import pymc3 as pm
b = 0.66 # Hellier 2011
period = 0.813475  # days # exo.mast.stsci.edu
u = [0]

oot_guess = np.median(np.r_[flux[:2*18], flux[-18:]])
stellar_variance = np.std(np.r_[flux[:2*18], flux[-18:]])
data = flux# - oot_guess

with pm.Model() as model:

    # The baseline flux
    mean_fwd = pm.Normal("mean_fwd", mu=1.0, sd=1.0)
    mean_rev = pm.Normal("mean_rev", mu=1.0, sd=1.0)

    # The time of a reference transit for each planet
    t0 = pm.Normal("t0", mu=t0_guess, sd=1e-6)  # , shape=2)

    # The log period; also tracking the period itself
    # logP = pm.Normal("logP", mu=np.log(periods), sd=0.1, shape=2)
    # period = pm.Deterministic("period", pm.math.exp(logP))

    # The Kipping (2013) parameterization for quadratic limb darkening paramters
    # u = xo.distributions.QuadLimbDark("u", testval=np.array([0.3, 0.2]))

    edepth = pm.Uniform("edepth", lower=1e-6, upper=0.1)  # , shape=2)  #, testval=np.array([0.04, 0.06]))
    edepth = np.sqrt(edepth)
    # b = xo.distributions.ImpactParameter("b", ror=r, shape=2, testval=np.random.rand(2))

    slope = pm.Uniform("slope", lower=-0.1, upper=0.1)
    # intercept = pm.Uniform("intercept", lower=-0.1, upper=0.1)
    line = slope * (t-t0_guess) + mean_fwd

    # Set up a Keplerian orbit for the planets
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0s, b=b)

    # # Compute the model light curve using starry
    light_curves = xo.LimbDarkLightCurve(u).get_light_curve(orbit=orbit, r=edepth, t=t)
    light_curve = pm.math.sum(light_curves, axis=-1) 

    # # Here we track the value of the model light curve for plotting
    # # purposes
    pm.Deterministic("light_curves", light_curves)

    # # In this line, we simulate the dataset that we will fit
    # # y = xo.eval_in_model(light_curve)
    # # y += yerr * np.random.randn(len(y))

    # # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=light_curve + line, sd=yerr, observed=data)

    # # Fit for the maximum a posteriori parameters given the simuated dataset
    map_soln = xo.optimize(start=model.test_point)

Now we can plot the simulated data and the maximum a posteriori model to make sure that our initialization looks ok.

In [ ]:
print(f"Found an eclipse of size {map_soln['edepth']*1e6:.0f} ppm at {map_soln['edepth']*86400:.2f} seconds from expected")

In [ ]:
np.where(map_soln["light_curves"] < 0)[0]

In [ ]:
plt.errorbar(t, data, yerr, color="k", fmt='o', ms=4, label="data")
plt.plot(t, map_soln["light_curves"] + map_soln['mean'], lw=1)
plt.axhline(0.0, ls='--', color='orange')
plt.xlim(t.min(), t.max())
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.legend(fontsize=10)
plt.title("map model");

## Sampling

Now, let's sample from the posterior defined by this model.
As usual, there are strong covariances between some of the parameters so we'll use :func:`exoplanet.get_dense_nuts_step`.

In [ ]:
np.random.seed(42)
with model:
    trace = pm.sample(
        tune=3000,
        draws=3000,
        start=map_soln,
        chains=4,
        step=xo.get_dense_nuts_step(target_accept=0.9),
        cores=mp.cpu_count()
    )a

After sampling, it's important that we assess convergence.
We can do that using the `pymc3.summary` function:

In [ ]:
pm.summary(trace, varnames=["t0", "edepth", "mean", "slope"])

That looks pretty good!
Fitting this without *exoplanet* would have taken a lot more patience.

Now we can also look at the [corner plot](https://corner.readthedocs.io) of some of that parameters of interest:

In [ ]:
import corner

samples = pm.trace_to_dataframe(trace, varnames=["t0", "edepth", "mean", "slope"])
truth = [t0_guess, 0.0, 1.0, 0.0]
corner.corner(samples, truths=truth, labels=["t0", "edepth", "mean", "slope"]);

## Phase plots

Like in the radial velocity tutorial (:ref:`rv`), we can make plots of the model predictions for each planet.

In [ ]:
plt.figure()

# Get the posterior median orbital parameters
p = period
t0 = np.median(trace["t0"])

# Plot the folded data
plt.errorbar(t - t0, data, yerr=yerr, fmt=".k", label="data", zorder=-1000)

# Plot the folded model
preds = trace["light_curves"][:,:,0] + trace["mean"][:, None]
pred = np.median(preds, axis=0)
# pred -= pred.min()
plt.plot(t - t0, pred, color="C1", label="model", zorder=10)
plt.axhline(0.0, ls='--', color='k')

# Annotate the plot with the planet's period
txt = f"Eclipse Depth = {np.mean(trace['edepth']*1e6):.0f}"
txt += f" +/- {np.std(trace['edepth']*1e6):.0f} ppm"

plt.annotate(
    txt,
    (0, 0),
    xycoords="axes fraction",
    xytext=(5, 5),
    textcoords="offset points",
    ha="left",
    va="bottom",
    fontsize=12,
)

add_traces = False
if add_traces:
  n_traces = 1000
  idx_rand = np.random.choice(np.arange(preds.shape[0]), size=n_traces, replace=False)
  for pred_ in preds[idx_rand]:
      plt.plot(t - t0, pred, color="grey", alpha=0.5, zorder=0)

plt.legend(fontsize=10, loc=4)
plt.xlim((t - t0).min(), (t - t0).max())
plt.xlabel("Time Since Eclipse [days]")
plt.ylabel("Relative Flux")
plt.title("PlanetName UVIS Eclipse");

## Citations

As described in the :ref:`citation` tutorial, we can use :func:`exoplanet.citations.get_citations_for_model` to construct an acknowledgement and BibTeX listing that includes the relevant citations for this model.
This is especially important here because we have used quite a few model components that should be cited.

In [ ]:
with model:
    txt, bib = xo.citations.get_citations_for_model()
print(txt)

In [ ]:
print("\n".join(bib.splitlines()[:10]) + "\n...")